In [43]:
from bs4 import BeautifulSoup, Comment
import re
import requests, time, logging, json
import uuid
import tldextract
from openai import OpenAI
from dotenv import load_dotenv
import os
from typing import Dict, Any

load_dotenv("config.env")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def get_openai_response(messages,response_format = "text"):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                   "role": "user",
                   "content": [
                       {
                           "type": "text",
                           "text": messages
                       }
                   ]
                }
            ],
            response_format={
                "type": response_format
            }
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"调用OpenAI API出错: {e}")
        return None

class Website_agent:
    def __init__(self, domain, existing_urls=None):
        self.crawl_key = os.getenv("CRAWLBASE_KEY")
        if not existing_urls:
            self.existing_urls = [domain]
        else:
            self.existing_urls = existing_urls

        self.main_domain = domain
        if not domain.startswith(('http://', 'https://')):
            domain = 'https://' + domain
        self.domain = re.sub(r'/$', '', domain)

        self.logger = logging.getLogger(__name__)

    def generate_request_id(self):
        random_uuid = uuid.uuid4()
        return random_uuid

    def fetch_html_by_js_token(self, url, max_retry=2):
        api_url = f'https://api.crawlbase.com/?token={self.crawl_key}&url={url}'
        for i in range(max_retry):
            try:
                print(f"正在抓取页面: {url} (第{i + 1}次尝试)")
                response = requests.get(api_url)
                if response.status_code == 200:
                    print("页面抓取成功")
                    return response.text
                else:
                    print(f"页面请求失败，状态码: {response.status_code}，尝试重试...")
                    if i < max_retry - 1:
                        time.sleep(1)
            except requests.exceptions.RequestException as e:
                print(f"网络请求异常: {str(e)}，尝试重试...")
                if i < max_retry - 1:
                    time.sleep(1)
                else:
                    raise
        raise requests.exceptions.RequestException(f"达到最大重试次数，URL抓取失败: {url}")

    def extract_head_info(self, html):
        soup = BeautifulSoup(html, 'html.parser')
        head_tag = soup.head

        title = head_tag.title.string if head_tag.title else None

        description = head_tag.find('meta', attrs={'name': 'description'}).get('content') if head_tag.find('meta',
                                                                                                           attrs={
                                                                                                               'name': 'description'}) else None

        return title, description

    def extract_body_tags(self, html):
        soup = BeautifulSoup(html, 'html.parser')
        body_tag = soup.body

        if not body_tag:
            return False

        for script_tag in body_tag.find_all('script'):
            script_tag.extract()

        for noscript_tag in body_tag.find_all('noscript'):
            noscript_tag.extract()

        for style_tag in body_tag.find_all('style'):
            style_tag.extract()

        for link_tag in body_tag.find_all('link'):
            link_tag.extract()

        for img_tag in body_tag.find_all('img'):
            img_tag.attrs = {'alt': img_tag.get('alt', '')}

        for tag in body_tag.find_all(True):
            tag.attrs = {attr: value for attr, value in tag.attrs.items() if attr.lower() != 'style'}

        for svg_tag in body_tag.find_all('svg'):
            svg_tag.extract()

        for tag in body_tag.find_all(True):
            if tag.name == 'a':
                tag.attrs = {attr: value for attr, value in tag.attrs.items() if attr.lower() == 'href'}
            else:
                tag.attrs = {}

        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        body_tags = ''.join(str(tag) for tag in body_tag.contents)
        body_tags = body_tags.replace("\n", "")

        return body_tags

    def extract_and_process_values(self, text):
        status_pattern = r'<status>(.*?)</status>'
        url_pattern = r'<url>(.*?)</url>'
        answer_pattern = r'<answer>(.*?)</answer>'

        status_matches = re.findall(status_pattern, text, re.DOTALL)
        url_matches = re.findall(url_pattern, text)
        answer_matches = re.findall(answer_pattern, text, re.DOTALL)

        statuses = [match.strip() for match in status_matches]
        urls = []
        contacts = []
        for match in url_matches:
            match = match.strip()
            if match:
                if match.startswith('tel:') or match.startswith('mailto:'):
                    contacts.append(match)
                else:
                    if not match.startswith('http://') and not match.startswith('https://'):
                        if match.startswith('/'):
                            url_value = self.domain.rstrip('/') + match
                        else:
                            url_value = match
                    else:
                        url_value = match
                    url_value = re.sub(r'/$', '', url_value)
                    if url_value not in self.existing_urls:
                        match_domain = extract_main_domain(url_value)
                        if match_domain == self.main_domain:
                            urls.append(url_value)
                        else:
                            print(f"跳过外部域名 - 当前域名:{self.main_domain}, 匹配域名:{match_domain}")

        urls = list(set(urls))
        answers = [match.strip() for match in answer_matches]

        result = {
            'status': 0,
            'answer': '',
            'urls': [],
            'contacts': []
        }

        if 'Fully meets the query requirement' in statuses:
            result['status'] = 1
        elif 'Partially meets the query requirement' in statuses:
            result['status'] = 2
        elif 'Unable to meet the query requirement' in statuses:
            result['status'] = 3

        if answers:
            result['answer'] = ' '.join(answers)

        if urls:
            result['urls'] = urls

        if contacts:
            result['contacts'] = contacts

        return result

    def process_website(self, htmlObj, url, question):
        print(f"\n开始处理网页: {url}")
        if 'body' not in htmlObj or not htmlObj['body']:
            return {"status": 4, "answer": "", "urls": [], "error": "页面内容为空"}

        if 'title' not in htmlObj:
            htmlObj['title'] = ''
        if 'description' not in htmlObj:
            htmlObj['description'] = ''

        try:
            prompt = f"""###
            The URL to be crawled:{url}
            Title:{htmlObj['title']}
            Description:{htmlObj['description']}
            Information contained in the website's HTML body tags:
            {htmlObj['body']}
            ###

            Instructions:
            1. The content enclosed by ### above describes website content. When responding to a Question, provide answers based on this website content.
            2. There are three statuses for responding to questions (the provided URL is just an example, and the answer tag should only contain the response):
               a). <status>Fully meets the query requirement</status> <answer>The answer</answer>
               b). <status>Partially meets the query requirement</status>, with the potential for more detailed answers by crawling <urls><url>website1</url><url>website2</url></urls>, <answer>The answer</answer>
               c). <status>Unable to meet the query requirement</status>, if there are recommended URLs to crawl, please specify them in the format: <urls><url>website address</url><url>website address</url></urls> for the answer
            3. The suggested URLs for crawling should not be identical to the URL being crawled: {url}.
            4. When the question requires providing contact information, include email, phone number, social media, etc.
            5. If there is data in the website content that is relevant to the question, it's best to reflect that relevant data in the answer.    

            Question:
            {question}
            """
            print("正在分析页面内容...")
            response = get_openai_response(prompt)
            print("页面分析完成，开始提取结果...")
            result = self.extract_and_process_values(response)
            return result
        except Exception as e:
            print(f"处理页面时出现错误: {str(e)}")
            return {"status": 0, "answer": "", "urls": [], "error": str(e)}

    def fetch_html_and_process_website(self, url, question):
        try:
            print(f"\n获取并处理页面: {url}")
            html_content = self.fetch_html_by_js_token(url)
            title, description = self.extract_head_info(html_content)
            body_tags = self.extract_body_tags(html_content)

            if body_tags is False:
                return {"status": 4, "error": "页面内容为空"}

            prompt = f"""###
            The URL to be crawled:{url}
            Title:{title}
            Description:{description}
            Information contained in the website's HTML body tags:
            {body_tags}
            ###

            Instructions:
            1. The content enclosed by ### above describes website content. When responding to a Question, provide answers based on this website content.
            2. There are three statuses for responding to questions (the provided URL is just an example, and the answer tag should only contain the response):
               a). <status>Fully meets the query requirement</status> <answer>The answer</answer>
               b). <status>Partially meets the query requirement</status>, with the potential for more detailed answers by crawling <urls><url>website1</url><url>website2</url></urls>, <answer>The answer</answer>
               c). <status>Unable to meet the query requirement</status>, if there are recommended URLs to crawl, please specify them in the format: <urls><url>website address</url><url>website address</url></urls> for the answer
            3. The suggested URLs for crawling should not be identical to the URL being crawled: {url}.
            4. When the question requires providing contact information, include email, phone number, social media, etc.
            5. If there is data in the website content that is relevant to the question, it's best to reflect that relevant data in the answer.    

            Question:
            {question}
            """

            response = get_openai_response(prompt)
            result = self.extract_and_process_values(response)

            return result

        except Exception as e:
            print(f"处理页面时出现错误: {str(e)}")
            raise

    def process_website_url(self, htmlObj, url, question, request_id=None):
        print(f"\n提取页面URL: {url}")
        if 'body' not in htmlObj or not htmlObj['body']:
            return {"status": 4, "error": "页面内容为空"}

        if 'title' not in htmlObj:
            htmlObj['title'] = ''
        if 'description' not in htmlObj:
            htmlObj['description'] = ''

        try:
            prompt = f"""###
            The URL to be crawled:{url}
            Title:{htmlObj['title']}
            Description:{htmlObj['description']}
            Information contained in the website's HTML body tags:
            {htmlObj['body']}
            ###
            你是一名数据分析工程师。

            问题：{question}

            1.分析html并且提取所有有效网址以及描述信息
            2.根据提取的信息把与问题相关的网址分别列出来，并使用<url></url>标签标记出来。
            3.不相关的不需要标记。
            """

            response = get_openai_response(prompt)
            url_pattern = r'<url>(.*?)</url>'
            url_matches = re.findall(url_pattern, response)
            urls = []
            contacts = []
            for match in url_matches:
                match = match.strip()
                url_value = ""
                if match:
                    if match.startswith('tel:') or match.startswith('mailto:'):
                        contacts.append(match)
                    else:
                        if not match.startswith('http://') and not match.startswith('https://'):
                            if match.startswith('/'):
                                url_value = self.domain.rstrip('/') + match
                        else:
                            match_domain = extract_main_domain(match)
                            if match_domain == self.main_domain:
                                url_value = match
                            else:
                                print(f"跳过外部域名 - 当前域名:{self.main_domain}, 匹配域名:{match_domain}")
                        if url_value:
                            url_value = re.sub(r'/$', '', url_value)
                            if url_value not in self.existing_urls:
                                urls.append(url_value)

            urls = list(set(urls))

            if not request_id:
                request_id = self.generate_request_id()

            return urls

        except Exception as e:
            print(f"处理页面URL时出现错误: {str(e)}")
            return {"status": 0, "error": str(e)}

    def test_process_website(self, url, question, max_depth=3):
        request_id = self.generate_request_id()
        print(f"\n=== 开始网站分析 ===")
        print(f"初始URL: {url}")
        print(f"最大探索深度: {max_depth}")
        
        try:
            print("\n第1步: 获取页面内容")
            html_content = self.fetch_html_by_js_token(url)
            title, description = self.extract_head_info(html_content)
            body_tags = self.extract_body_tags(html_content)

            if not body_tags:
                print("错误: 页面内容为空")
                return {"request_id": request_id, "status": 4, "error": "页面内容为空"}

            print("\n第2步: 分析页面内容")
            htmlObj = {'title': title, 'description': description, 'body': body_tags}
            result = self.process_website(htmlObj, url, question)
            
            print(f"\n第3步: 处理分析结果")
            print(f"状态码: {result['status']}")
            if result['status'] == 1:
                print("✓ 已找到完整答案")
                return {'status': result['status'], 'answer': result['answer']}
            
            elif result['status'] == 2:
                print("→ 找到部分答案，需要进一步分析...")
                if max_depth > 0:
                    print("\n第4步: 分析相关页面")
                    return self._process_additional_urls(result, htmlObj, url, question, request_id, max_depth)
                else:
                    print("已达到最大深度，返回当前结果")
                    return {'status': 2, 'answer': result['answer']}
                    
            elif result['status'] == 3:
                print("→ 未找到答案，尝试分析其他页面...")
                if max_depth > 0:
                    print("\n第4步: 分析其他相关页面")
                    return self._process_additional_urls(result, htmlObj, url, question, request_id, max_depth)
                else:
                    print("已达到最大深度，无法找到答案")
                    return {'status': 3}
            
            return result

        except Exception as e:
            print(f"\n处理过程中出现错误: {str(e)}")
            return {"status": 0, "error": str(e)}

    def _process_additional_urls(self, initial_result, htmlObj, url, question, request_id, max_depth):
        urls = initial_result['urls'] or self.process_website_url(htmlObj, url, question, request_id)
        if urls:
            print(f"发现 {len(urls)} 个相关链接")
        answers = [initial_result['answer']] if initial_result['answer'] else []
        
        for i, url_value in enumerate(urls, 1):
            if url_value not in self.existing_urls:
                print(f"\n分析相关链接 {i}/{len(urls)}: {url_value}")
                self.existing_urls.append(url_value)
                sub_result = self.fetch_html_and_process_website(url_value, question)
                
                if sub_result['status'] == 1:
                    print("✓ 在相关页面中找到完整答案")
                    return sub_result
                elif sub_result['status'] == 2:
                    print("→ 找到部分相关信息")
                    answers.append(sub_result['answer'])

        if answers:
            print("\n整合所有找到的信息...")
            return {'status': 2, 'answer': '\n'.join(answers)}
        
        print("\n未能找到相关答案")
        return {'status': 3}

def extract_main_domain(url):
    if not url.startswith(('http://', 'https://')):
        url = 'https://' + url
    ext = tldextract.extract(url)
    return ".".join(part for part in (ext.domain, ext.suffix) if part)

In [18]:
# 测试代码
crawl_url = "https://www.percent.cn"
existing_urls = [crawl_url]
main_domain = extract_main_domain(crawl_url)
question = "主要成员以及联系方式"

website_bot = Website_agent(main_domain,existing_urls)
result = website_bot.test_process_website(crawl_url,question)

print("抓取结果:")
print(f"状态码: {result['status']}")

if result['status'] == 0:
    print(f"错误信息:")
else:
    if result['status'] == 2 or result['status'] == 1:
        print(f"答案: {result['answer']}")
    if result['status'] == 3:
        print("无法回答")
    if result['status'] == 4:
        print("无法回答")



=== 开始网站分析 ===
初始URL: https://www.percent.cn
最大探索深度: 3

第1步: 获取页面内容
正在抓取页面: https://www.percent.cn (第1次尝试)
页面抓取成功

第2步: 分析页面内容

开始处理网页: https://www.percent.cn
正在分析页面内容...
页面分析完成，开始提取结果...
跳过外部域名 - 当前域名:percent.cn, 匹配域名:linkedin.com
跳过外部域名 - 当前域名:percent.cn, 匹配域名:crunchbase.com

第3步: 处理分析结果
状态码: 2
→ 找到部分答案，需要进一步分析...

第4步: 分析相关页面

提取页面URL: https://www.percent.cn
发现 1 个相关链接

分析相关链接 1/1: https://percent.cn/Contact.html

获取并处理页面: https://percent.cn/Contact.html
正在抓取页面: https://percent.cn/Contact.html (第1次尝试)
页面抓取成功
→ 找到部分相关信息

整合所有找到的信息...
抓取结果:
状态码: 2
答案: 目前提供的内容没有涉及主要成员的信息。不过，您可以通过以下方式联系百分点科技：电话：400-6240-800，商务邮箱：business@percent.cn。
联系方式包括：
- 咨询电话：400-6240-800
- 商务合作邮箱：business@percent.cn
- 人才招聘邮箱：hr@percent.cn
- 媒体合作邮箱：pr@percent.cn
主要成员信息未在提供的网站内容中列出。


In [48]:
serper_api_key = os.getenv("SERPER_KEY")

def process_query(query: str) -> Dict[str, Any]:
    """
    分析用户查询并执行相应的技能
    Args:
        query: 用户的查询内容
    Returns:
        执行结果的字典
    """
    print(f"\n开始处理用户查询: {query}")
    
    prompt = f"""你是AI网络搜索问答助手，你的主要工作是根据问题来判断下一步工作应该触发哪个技能。

技能分类：
搜索官方网站 (type_code: search)
使用场景：当用户询问某个产品、公司或服务，但未提供官方URL
必须生成用于查找官方网站的搜索关键词
返回格式：{{"type_code": "search", "keyword": "公司名称 official website","question":"用户的问题"}}

网站采集 (type_code: crawl)
使用场景：用户直接提供了目标URL
返回格式：{{"type_code": "crawl", "url": "用户提供的URL","question":"用户的问题"}}

直接回答 (type_code: other)
使用场景：普通问答或闲聊
返回格式：{{"type_code": "other", "answer": "回答内容"}}

严格按照上述JSON格式返回结果，不要包含任何其他说明文字。

用户问题：{query}
"""

    # 获取GPT分析结果
    print("正在分析查询类型...")
    analysis_result = json.loads(get_openai_response(prompt, "json_object"))
    print(f"分析结果: {json.dumps(analysis_result, ensure_ascii=False, indent=2)}")

    # 根据分析结果执行相应技能
    if analysis_result['type_code'] == 'search':
        print(f"\n执行搜索技能")
        print(f"搜索关键词: {analysis_result['keyword']}")
        return search_and_process(analysis_result['question'])
            
    elif analysis_result['type_code'] == 'crawl':
        print(f"\n执行网站采集")
        crawl_url = analysis_result['url']
        existing_urls = [crawl_url]
        main_domain = extract_main_domain(crawl_url)
        website_bot = Website_agent(main_domain, existing_urls)
        result = website_bot.test_process_website(crawl_url, analysis_result['question'])
        
        print("抓取结果:")
        print(f"状态码: {result['status']}")
        if result['status'] == 0:
            print(f"错误信息:")
        else:
            if result['status'] == 2 or result['status'] == 1:
                print(f"答案: {result['answer']}")
            if result['status'] == 3:
                print("无法回答")
            if result['status'] == 4:
                print("无法回答")
                
        return result
        
    else:  # type_code == 'other'
        print("\n执行直接回答")
        return analysis_result

def build_official_site_prompt(search_results: list) -> str:
    """
    构建用于判断官方网站的prompt
    Args:
        search_results: Google搜索结果列表
    """
    print("\n=== 构建官网识别Prompt ===")
    
    prompt = """你是一个网站分类专家。你的任务是从搜索结果中识别出最可能的官方网站。
    
规则：
1. 官方网站通常具有以下特征：
   - 域名通常是公司/品牌名称
   - URL结构简单，通常是根域名
   - 网站标题通常包含公司/品牌名称
   - 一般不会是社交媒体、新闻网站或第三方平台的链接

2. 需要考虑的因素（按重要性排序）：
   a) 域名与公司/品牌名称的匹配度
   b) 是否是品牌的主域名（而不是子域名）
   c) 链接的可信度
   d) 网站描述的官方性

请分析以下搜索结果，并以JSON格式返回最可能的官方网站URL。

搜索结果：
"""
    # 添加搜索结果到prompt
    print(f"\n分析以下搜索结果：")
    for idx, result in enumerate(search_results, 1):
        url = result.get('link', '')
        title = result.get('title', '')
        snippet = result.get('snippet', '')
        
        prompt += f"""
{idx}. 网站信息：
   URL: {url}
   标题: {title}
   描述: {snippet}
"""
        print(f"\n{idx}. {url}")
        print(f"   标题: {title}")

    prompt += """
请以以下JSON格式返回分析结果：
{
    "official_url": "最可能的官方网站URL",
    "confidence": "高/中/低",
    "reason": "简要说明选择原因"
}

注意：
- 如果没有找到明显的官方网站，confidence设为"低"
- 如果发现多个可能的官方网站，选择最可能的一个
"""
    print("\nPrompt构建完成")
    return prompt

def process_search_results(search_results: dict, question: str) -> dict:
    """
    处理Google搜索结果，找出官方网站
    Args:
        search_results: Google Serper的完整搜索结果
        question: 原始问题
    Returns:
        dict: 包含官方网站URL的结果
    """
    print("\n=== 开始处理搜索结果 ===")
    try:
        if 'organic' not in search_results:
            print("❌ 错误：搜索结果格式错误")
            return {"error": "搜索结果格式错误"}

        print(f"找到 {len(search_results['organic'])} 个搜索结果")
        
        # 构建分析prompt
        print("\n第1步：构建分析Prompt")
        prompt = build_official_site_prompt(search_results['organic'][:5])  # 只分析前5个结果
        
        # 分析结果
        print("\n第2步：使用AI分析搜索结果")
        try:
            result = json.loads(get_openai_response(prompt, "json_object"))
            
            print(f"\n=== AI分析结果 ===")
            print(f"✓ 识别出的官方网站: {result['official_url']}")
            print(f"✓ 置信度: {result['confidence']}")
            print(f"✓ 选择原因: {result['reason']}")
            
            if result['official_url']:
                if result['confidence'] == "低":
                    print("\n警告：官网识别置信度较低，但仍继续处理")
                    
                print(f"\n第3步：开始网站采集")
                print(f"目标网址: {result['official_url']}")
                
                # 实例化Website_agent并执行采集
                crawl_url = result['official_url']
                existing_urls = [crawl_url]
                main_domain = extract_main_domain(crawl_url)
                
                agent = Website_agent(main_domain,existing_urls)
                return agent.test_process_website(
                    url=crawl_url,
                    question=question
                )
            else:
                print("\n❌ 错误：未找到可信的官方网站")
                return {"error": "未找到可信的官方网站"}
                
        except Exception as e:
            print(f"\n❌ 错误：分析过程失败")
            print(f"错误信息: {str(e)}")
            return {"error": f"分析过程发生错误: {str(e)}"}
            
    except Exception as e:
        print(f"\n❌ 错误：处理搜索结果失败")
        print(f"错误信息: {str(e)}")
        return {"error": f"处理搜索结果时发生错误: {str(e)}"}

def search_and_process(query: str) -> dict:
    """搜索并处理结果"""
    print("\n=== 开始搜索流程 ===")
    search_keyword = f"{query.strip()} official website"
    print(f"搜索关键词: {search_keyword}")
    
    headers = {
        'X-API-KEY': serper_api_key,
        'Content-Type': 'application/json'
    }
    payload = {
        'q': search_keyword,
        'gl': 'us',
        'page': 0,
        'num': 5
    }
    
    try:
        print("\n第1步：调用搜索API")
        response = requests.post(
            "https://google.serper.dev/search",
            headers=headers,
            json=payload
        )
        
        if response.status_code == 200:
            print("✓ 搜索请求成功")
            search_results = response.json()
            
            print("\n第2步：处理搜索结果")
            return process_search_results(search_results, query)
        else:
            print(f"\n❌ 错误：搜索请求失败 (状态码: {response.status_code})")
            return {"error": f"搜索请求失败: {response.status_code}"}
            
    except Exception as e:
        print(f"\n❌ 错误：搜索过程失败")
        print(f"错误信息: {str(e)}")
        return {"error": f"搜索过程发生错误: {str(e)}"}

In [56]:
#result2 = search_and_process(query2)

query2 = "https://www.percent.cn/百分点公司主要成员以及联系方式"
#query2 = "百分点公司主要成员以及联系方式"
#query2 = "今天天气怎么样"
result2 = process_query(query2)
print(f"查询问题：{query2}")
print("\n最终结果：")
print(result2)


开始处理用户查询: https://www.percent.cn/百分点公司主要成员以及联系方式
正在分析查询类型...
分析结果: {
  "type_code": "crawl",
  "url": "https://www.percent.cn/",
  "question": "百分点公司主要成员以及联系方式"
}

执行网站采集

=== 开始网站分析 ===
初始URL: https://www.percent.cn/
最大探索深度: 3

第1步: 获取页面内容
正在抓取页面: https://www.percent.cn/ (第1次尝试)
页面抓取成功

第2步: 分析页面内容

开始处理网页: https://www.percent.cn/
正在分析页面内容...
页面分析完成，开始提取结果...

第3步: 处理分析结果
状态码: 2
→ 找到部分答案，需要进一步分析...

第4步: 分析相关页面

提取页面URL: https://www.percent.cn/
发现 2 个相关链接

分析相关链接 1/2: https://www.percent.cn/Company.html

获取并处理页面: https://www.percent.cn/Company.html
正在抓取页面: https://www.percent.cn/Company.html (第1次尝试)
页面抓取成功
→ 找到部分相关信息

分析相关链接 2/2: https://www.percent.cn/Contact.html

获取并处理页面: https://www.percent.cn/Contact.html
正在抓取页面: https://www.percent.cn/Contact.html (第1次尝试)
页面抓取成功
→ 找到部分相关信息

整合所有找到的信息...
抓取结果:
状态码: 2
答案: 目前从所提供的内容中无法获取百分点公司的主要成员信息。不过，您可以联系他们获取更多信息。以下是他们的联系方式：
- 咨询电话：400-6240-800
- 商务邮箱：business@percent.cn
- 更多联系方式可以访问他们的联系页面：<a href="https://www.percent.cn/Contact.html">更多联系方式</